In [1]:
import random
import time

In [50]:
import tensorflow as tf
import numpy as np

import zmq

from tensorflow.python.training import moving_averages

In [102]:
import learner
reload(learner)

<module 'learner' from 'learner.py'>

In [4]:
exp_socket_addr = "tcp://127.0.0.1:19884" 
max_actions = 7
strat_socket_addr = "tcp://127.0.0.1:19885"

In [5]:
context = zmq.Context()
sock_exp = context.socket(zmq.REP)
sock_exp.bind(exp_socket_addr)

In [6]:
sock_strat = context.socket(zmq.PUB)
sock_strat.bind(strat_socket_addr)

In [7]:
NUM_HIDDEN = 128

In [51]:
def CreateNetwork(state, num_actions, scope, is_training, reuse=False):
    with tf.variable_scope(scope, reuse=reuse):
#         moving_mean = tf.get_variable('mean', shape=state.get_shape()[1],
#                             initializer=tf.zeros_initializer,
#                             trainable=False)
#         moving_var = tf.get_variable('var', shape=state.get_shape()[1],
#                             initializer=tf.ones_initializer, 
#                             trainable=False)
#         state = (state - moving_mean) / tf.sqrt(moving_var + 0.001)
        
        hidden1 = tf.contrib.layers.fully_connected(
            state, NUM_HIDDEN,
            activation_fn=tf.nn.elu,
            weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
#             weights_initializer=tf.contrib.layers.xavier_initializer(),
            biases_initializer=tf.constant_initializer(0.01),
            scope='hidden1', reuse=reuse)
        hidden2 = tf.contrib.layers.relu(
            hidden1, NUM_HIDDEN,
            activation_fn=tf.nn.elu,
            weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
#             weights_initializer=tf.contrib.layers.xavier_initializer(),
            biases_initializer=tf.constant_initializer(0.01),
            scope='hidden2', reuse=reuse)

    #     value_hid = tf.contrib.layers.relu(hidden2, NUM_HIDDEN,
    #         weights_initializer=tf.truncated_normal_initializer(stddev=0.1),
    #         biases_initializer=tf.constant_initializer(0.01),
    #         scope=scope + '/val_hid', reuse=reuse)

    #     adv_hid = tf.contrib.layers.relu(hidden2, NUM_HIDDEN,
    #         weights_initializer=tf.truncated_normal_initializer(stddev=0.1),
    #         biases_initializer=tf.constant_initializer(0.01),
    #         scope=scope + '/adv_hid', reuse=reuse)

        value = tf.contrib.layers.linear(hidden2, 1,
                                         weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
#                                          weights_initializer=tf.contrib.layers.xavier_initializer(),
                                         biases_initializer=tf.constant_initializer(0.),
                                         scope='value')
        adv = tf.contrib.layers.linear(hidden2, num_actions,
                                       weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
#                                        weights_initializer=tf.contrib.layers.xavier_initializer(),
                                       scope='advantage')
        adv = tf.sub(adv, tf.reduce_mean(adv, reduction_indices=1, keep_dims=True), 'advantage')

        output = tf.add(value, adv, 'output')
    #     return hidden1, hidden2, value_hid, adv_hid, value, adv, output
#         return moving_mean, moving_var, state, hidden1, hidden2, value, adv, output
    return hidden1, hidden2, value, adv, output

In [52]:
def Select(value, index):
    # Value - float tensor of (batch, actions) size
    # index - int32 tensor of (batch) size
    # returns float tensor of batch size where in every batch the element from index is selected
    batch_size = tf.shape(value)[0]
    _range = tf.range(0, batch_size)
    ind = tf.concat(1, [tf.expand_dims(_range, 1), 
                        tf.expand_dims(index, 1)])
    return tf.gather_nd(value, ind)

In [53]:
def Select4(value, index):
    # Value - float tensor of (batch, actions) size
    # index - int32 tensor of (batch) size
    # returns float tensor of batch size where in every batch the element from index is selected
    shp = tf.shape(value)
    return tf.reduce_sum(value * tf.one_hot(index, shp[1]), reduction_indices=1)

In [54]:
DEFAULT_OPTIONS = {
    'clip_grad': 3.,
    'learning_rate': 0.001,
    'update_steps': 10000,
}

In [55]:
# Compute the normalization coefficients for state
size = min(buf.inserted, buf.buffer_size)
STATE_MEAN, STATE_STD = buf.ss[:, :size].mean(1), buf.ss[:, :size].std(1) + 0.01

In [112]:
a = 'asdfadsfasdf'

In [116]:
class QLearner(object):
    def __init__(self, exp_buffer, num_actions, run_index, options=DEFAULT_OPTIONS):
        self.exp_buffer = exp_buffer
        self.num_actions = num_actions
        self.run_index = run_index
        self.options = options

        self.state = tf.placeholder(tf.float32, shape=[None, self.exp_buffer.state_size], name='state')
        self.action = tf.placeholder(tf.int32, shape=[None], name='action')
        self.reward = tf.placeholder(tf.float32, shape=[None], name='reward')
        self.state1 = tf.placeholder(tf.float32, shape=[None, self.exp_buffer.state_size], name='state1')
        self.gamma = tf.placeholder(tf.float32, shape=[None], name='gamma')
        self.is_weights = tf.placeholder(tf.float32, shape=[None], name='is_weights')       
        self.is_training = tf.placeholder(tf.bool, shape=None, name='is_training')

        mean = tf.get_variable('mean', dtype=tf.float32,
                               initializer=STATE_MEAN, trainable=False)
        std = tf.get_variable('std', dtype=tf.float32,
                              initializer=STATE_STD, trainable=False)       
        
#         norm_state = (self.state - self.moving_mean) / tf.sqrt(self.moving_var + 0.001)
#         norm_state1 = (self.state1 - self.moving_mean) / tf.sqrt(self.moving_var + 0.001)
        norm_state = (self.state - tf.identity(mean)) / tf.identity(std)
        norm_state1 = (self.state1 - mean) / std

        self.pred_vars = CreateNetwork(norm_state, num_actions, 'model', self.is_training)
        self.pred_vars_s1 = CreateNetwork(norm_state1, num_actions, 'model', self.is_training, True)
        self.target_vars = CreateNetwork(norm_state1, num_actions, 'target', False)

#         self.mean, self.var = tf.nn.moments(self.state, [0])
#         self.mean_update = moving_averages.assign_moving_average(self.pred_vars[0], self.mean, 0.99)
#         self.var_update = moving_averages.assign_moving_average(self.pred_vars[1], self.var, 0.99)
        
        self.vars_pred = tf.get_collection(tf.GraphKeys.VARIABLES, 'model')
        self.vars_target = tf.get_collection(tf.GraphKeys.VARIABLES, 'target')
        
        self.copy_op = tf.group(
            *[tf.assign(y, x) for x, y in zip(self.vars_pred, self.vars_target)]
        )
        self.vars_pred.extend([mean, std])

        idx = len(self.pred_vars) - 1

        self.act_s1 = tf.cast(tf.argmax(self.pred_vars_s1[idx], dimension=1), tf.int32)
        self.q_s1 = Select(self.target_vars[idx], self.act_s1)
        self.target_q = tf.stop_gradient(self.reward + self.gamma * self.q_s1)
        self.q = Select4(self.pred_vars[idx], self.action)

        self.delta = self.target_q - self.q
        self.td_err_weight = tf.abs(self.delta)
        self.loss = tf.reduce_mean(learner.HuberLoss(self.delta, 5) * self.is_weights)

        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

        self.global_step = tf.Variable(0, name='global_step', trainable=False)
        
        optimizer = tf.train.AdamOptimizer(options['learning_rate'])
        grads = optimizer.compute_gradients(self.loss, tf.get_collection(
                tf.GraphKeys.TRAINABLE_VARIABLES, 'model'))
        if 'clip_grad' in options:
            grads = [(tf.clip_by_norm(g, options['clip_grad']) if g is not None else None, v)
                     for g, v in grads]

        for grad, v in grads:
            if grad is not None:
                tf.histogram_summary('{}/grad'.format(v.name), grad)
                
        for v in tf.get_collection(tf.GraphKeys.VARIABLES):
            tf.histogram_summary(v.name, v)

#         for v in self.pred_vars[:2]:
#             tf.histogram_summary(v.name, v)

        for ten in self.pred_vars:
            tf.histogram_summary('vars/' + ten.name, ten)

        tf.histogram_summary('Monitor/TD Error', self.delta)
        tf.histogram_summary('Monitor/Q', self.q)
        tf.histogram_summary('Monitor/Weights', self.is_weights)
        tf.scalar_summary("Scalars/Loss", self.loss)
        tf.scalar_summary("Scalars/Q Func", tf.reduce_mean(self.q))
        tf.scalar_summary('Scalars/Weights', tf.reduce_mean(self.is_weights))
            
        self.train_op = optimizer.apply_gradients(grads, self.global_step)
#         if update_ops:
        self.train_op = tf.group(#tf.add_check_numerics_ops(), 
                                 self.train_op, #self.mean_update, self.var_update,
                                 *update_ops)

        self.summary_op = tf.merge_all_summaries()
        self.writer = None
        self.saver = None
        self.cur_step = None

    def step(self, sess, batch_size=32):
        idx, ss, aa, rr, ss1, gg, ww = self.exp_buffer.sample(batch_size)
        if ss is None:
            return
        
        if self.writer is None:
            self.writer = tf.train.SummaryWriter(#'/Users/vertix/tf/tensorflow_logs/aicup/%s'
                                                 '/media/vertix/UHDD/tmp/tensorflow_logs/aicup/%s'
                                                 % self.run_index)
            self.saver = tf.train.Saver(self.vars_pred)
            self.last_start = time.time()

        feed_dict = {self.state: ss, self.action: aa, self.reward: rr, self.state1:ss1,
                     self.gamma: gg, self.is_weights: ww,
                     self.is_training: True}

        if self.cur_step and self.cur_step % 100 != 0:
            self.cur_step, weights, _ = sess.run(
                [self.global_step, self.td_err_weight, self.train_op], feed_dict)
        else:
            self.cur_step, weights, _, smr = sess.run(
                [self.global_step, self.td_err_weight, self.train_op, self.summary_op], feed_dict)
            self.writer.add_summary(smr, self.cur_step)
        
        for ii, td_w in zip(idx, weights):
            self.exp_buffer.tree_update(ii, td_w)
            
        if self.cur_step % self.options['update_steps'] == 0:
            print 'Updated target network'
            sess.run(self.copy_op)
            self.saver.save(sess, self.run_index, global_step=self.global_step)
            if self.last_start is not None:
                self.writer.add_summary(
                    tf.Summary(
                        value=[tf.Summary.Value(
                                tag='Steps per sec',
                                simple_value=self.options['update_steps'] / (time.time() - self.last_start))]),
                    self.cur_step)
            self.last_start = time.time()

    def stat(self, data):
        self.writer.add_summary(
            tf.Summary(
                value=[tf.Summary.Value(tag=name, simple_value=value)
                       for name, value in data.items()]), self.cur_step)

In [103]:
buf = learner.WeightedExperienceBuffer(0.6, 0.4, 100, buffer_size=1<<18)
for _ in range(2):
    msg = sock_exp.recv_pyobj()
    sock_exp.send('Ok')
    if msg['type'] == 'exp':
        msg = msg['data']
        buf.add(msg['s'], msg['a'], msg['r'], msg['s1'], msg['g'], 100)

In [128]:
buf.inserted, buf.buffer_size

(1420, 262144)

In [45]:
for i in xrange(min(buf.buffer_size, buf.inserted)):
    buf.tree_update(i, 100)

In [63]:
buf.beta = 0.7

In [108]:
STATE_STD = STATE_STD.astype(np.float32)

In [117]:
sess.close()
tf.reset_default_graph()
sess = tf.InteractiveSession(config=tf.ConfigProto(device_count = {'GPU': 0}))

In [118]:
ql = QLearner(buf, 2 + 5, run_index='ql-2-fix-norm', options={
    'clip_grad': 3.,
    'learning_rate': 0.0001,
    'update_steps': 20000,
})
sess.run(tf.initialize_all_variables())

In [32]:
for _ in xrange(105):
    ql.step(sess)

1. Prioritized replay
2. Hanlde death by modifying gamma
3. Test Q-learning on simple task
4. Combine updates in batches

In [119]:
for v in ql.vars_pred:
    print v.name

model/hidden1/weights:0
model/hidden1/biases:0
model/hidden2/weights:0
model/hidden2/biases:0
model/value/weights:0
model/value/biases:0
model/advantage/weights:0
model/advantage/biases:0
mean:0
std:0


In [120]:
import threading

In [121]:
def FillBuffer(ql, coord):
    while not coord.should_stop():
        msg = sock_exp.recv_pyobj()
        sock_exp.send('Ok')
    
        if msg['type'] == 'exp':
            msg = msg['data']
            ql.exp_buffer.add(msg['s'], msg['a'], msg['r'], msg['s1'], msg['g'], 100)
        elif msg['type'] == 'stat':
            ql.stat(msg['data'])        

In [122]:
def Learn(ql, coord, sess):
    i = 0
    while not coord.should_stop():
        ql.step(sess)
        
        if i > 0 and i % 500 == 0:
            sock_strat.send_pyobj({v.name: sess.run(v, {ql.is_training: False})
                                   for v in ql.vars_pred})
        i += 1

In [267]:
Learn(ql, coord, sess)

KeyboardInterrupt: 

In [123]:
coord = tf.train.Coordinator()

In [124]:
threads = [threading.Thread(target=FillBuffer, args=(ql, coord)),
           threading.Thread(target=Learn, args=(ql, coord, sess))]

In [125]:
threads[0].start()
threads[1].start()

In [78]:
coord.request_stop()

In [126]:
ql.cur_step

1823

In [14]:
def FillBufferSimple(coord, buf):
    while not coord.should_stop():
        msg = sock_exp.recv_pyobj()
        sock_exp.send('Ok')
    
        if msg['type'] == 'exp':
            msg = msg['data']
            buf.add(msg['s'], msg['a'], msg['r'], msg['s1'], msg['g'], 100)

In [61]:
_t = threading.Thread(target=FillBufferSimple, args=(coord, buf))
_t.start()

-------------------
Use OpenAI env to test Q-Learning

In [11]:
import gym

In [19]:
env = gym.make('LunarLander-v2')

[2016-11-23 18:50:09,140] Making new env: LunarLander-v2


In [13]:
buf = WeightedExperienceBuffer()
old_s = env.reset()
for _ in range(5):
    a = env.action_space.sample()
    
    s, r, d, _ = env.step(a)
    buf.add(old_s, a, r, s if not d else None, TD_ERR_CLIP)
    old_s = s

In [15]:
sess.close()
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [16]:
ql = QLearner(buf, env.action_space.n, 8)
# feed_dict = {ql.state: ss, ql.action: aa, ql.reward: rr, ql.state1:ss1, ql.gamma: gg}
sess.run(tf.initialize_all_variables())

In [20]:
# %%prun

old_s = env.reset()
episode_rew = 0.
episode_len = 0.

for i in range(10000000):
    if i % 1 == 0:
        epsilon = 1.0 / (1 + (ql.cur_step or 0.) / 1000.)
        if np.random.sample() < epsilon:
            a = env.action_space.sample()
        else:
            a = sess.run(ql.act_s1, {ql.state1: np.reshape(old_s, (1, -1)),
                                     ql.is_training: False})
            a = a[0]

        s, r, d, _ = env.step(a)
        buf.add(old_s, a, r, s if not d else None, TD_ERR_CLIP)
        
        episode_rew += r
        episode_len += 1
        
        if d:
            ql.stat({'Env/Reward': episode_rew, 'Env/Lenght': episode_len})
            episode_rew, episode_len = 0., 0.
            old_s = env.reset()
        else:
            old_s = s

    ql.step(sess)

Updated target network
Updated target network
Updated target network
Updated target network
Updated target network
Updated target network
Updated target network
Updated target network
Updated target network
Updated target network


KeyboardInterrupt: 

In [27]:
old_s = env.reset()
done = False
r = 0
while not done:
#     env.render('human')
#     print old_s
    a = sess.run(ql.act_s1, {ql.state1: np.reshape(old_s, (1, -1)),
                             ql.is_training: False})
    old_s, reward, done, _ = env.step(a[0])
    r += reward
    
    if r < -1000:
        break
    
print r

-1001.0


----------------

In [81]:
network_vars = {v.name: v.eval() for v in ql.vars_pred}

In [82]:
sorted(network_vars.keys())

[u'model/advantage/biases:0',
 u'model/advantage/weights:0',
 u'model/hidden1/biases:0',
 u'model/hidden1/weights:0',
 u'model/hidden2/biases:0',
 u'model/hidden2/weights:0',
 u'model/value/biases:0',
 u'model/value/weights:0']

In [181]:
def Elu(x):
    return np.where(x < 0, np.exp(x) - 1, x)

In [182]:
def ReLu(x):
    return np.maximum(x, 0)

In [183]:
def BatchNorm(state, network_vars, key, shift=True, eps=0.001):
    inv = 1.0 / np.sqrt(network_vars[key + '/moving_variance:0'] + eps)

    if shift:
        return state * inv + (network_vars[key + '/beta:0'] - network_vars[key + '/moving_mean:0'] * inv)
    else:
        return state * inv + (-network_vars[key + '/moving_mean:0'] * inv)

In [200]:
class QFunction(object):
    def __init__(self, network_vars):
        self.vars = network_vars

    def Q(self, state):
#         state = Normalize(state, self.vars['model/mean:0'], self.vars['model/var:0'])
        state = BatchNorm(state, self.vars, 'model/norm', shift=False)

        state = np.matmul(state, self.vars['model/hidden1/weights:0'])
        state += self.vars['model/hidden1/biases:0']
        # state = BatchNorm(state, self.vars, 'model/hidden1/BatchNorm')
        state = Elu(state)

        state = np.matmul(state, self.vars['model/hidden2/weights:0'])
        state += self.vars['model/hidden2/biases:0']
        # state = BatchNorm(state, self.vars, 'model/hidden2/BatchNorm')
        state = Elu(state)

        # value = np.matmul(state, self.vars['model/val_hid/weights:0'])
        # value += self.vars['model/val_hid/biases:0']
        # value = BatchNorm(value, self.vars, 'model/val_hid/BatchNorm')
        # value = ReLu(value)
        value = np.matmul(state, self.vars['model/value/weights:0'])
        value += self.vars['model/value/biases:0']

        # adv = np.matmul(state, self.vars['model/adv_hid/weights:0'])
        # adv += self.vars['model/adv_hid/biases:0']
        # adv = BatchNorm(adv, self.vars, 'model/adv_hid/BatchNorm')
        # adv = ReLu(adv)
        adv = np.matmul(state, self.vars['model/advantage/weights:0'])
        adv += self.vars['model/advantage/biases:0']

        return value + (adv - adv.mean(1, keepdims=True))

In [116]:
_, ss, aa, rr, ss1, gg, ww = ql.exp_buffer.sample(10)

In [201]:
ql.pred_vars[5].eval({ql.state: ss, ql.is_training: False})

array([[ -6.92744926e-03,  -1.11189019e-02,  -1.30229145e-02,
          1.83991902e-03,  -1.30038373e-02,  -7.98689015e-03,
         -7.29485415e-04],
       [ -9.82989091e-04,  -1.13519998e-02,  -6.05490245e-03,
          8.68432131e-03,  -6.10211212e-03,   1.47880614e-03,
         -2.00587325e-03],
       [ -2.23232387e-03,  -1.66344829e-02,  -1.05185825e-02,
          3.23185511e-03,  -8.59550014e-03,  -6.28848234e-03,
          1.69825740e-04],
       [ -8.51184595e-04,  -1.09778065e-02,  -3.41447210e-03,
          1.37389638e-03,  -6.63597416e-03,  -1.76197849e-04,
          4.25715977e-03],
       [  5.10341488e-06,  -1.06490199e-02,  -2.82861246e-03,
          1.96091714e-03,  -5.35972510e-03,  -8.95796227e-04,
          1.86092989e-03],
       [  1.64952967e-03,  -6.31430326e-03,  -1.34359929e-04,
          2.89097405e-03,  -7.24052545e-03,   4.00833832e-03,
          1.31299091e-03],
       [ -2.26311595e-03,  -1.39367068e-02,  -1.26700234e-02,
          4.39036638e-03,  -1.01

In [202]:
QFunction(network_vars).Q(ss)

array([[ -6.92745066e-03,  -1.11189029e-02,  -1.30229194e-02,
          1.83991885e-03,  -1.30038356e-02,  -7.98689320e-03,
         -7.29486923e-04],
       [ -9.82990431e-04,  -1.13520040e-02,  -6.05490711e-03,
          8.68431824e-03,  -6.10211620e-03,   1.47880294e-03,
         -2.00587536e-03],
       [ -2.23232535e-03,  -1.66344838e-02,  -1.05185814e-02,
          3.23185655e-03,  -8.59550059e-03,  -6.28848224e-03,
          1.69828431e-04],
       [ -8.51183788e-04,  -1.09778067e-02,  -3.41447230e-03,
          1.37389636e-03,  -6.63597596e-03,  -1.76202095e-04,
          4.25715998e-03],
       [  5.10407696e-06,  -1.06490186e-02,  -2.82861120e-03,
          1.96091833e-03,  -5.35972460e-03,  -8.95791789e-04,
          1.86093314e-03],
       [  1.64952601e-03,  -6.31430388e-03,  -1.34360217e-04,
          2.89097393e-03,  -7.24052295e-03,   4.00833684e-03,
          1.31298873e-03],
       [ -2.26311739e-03,  -1.39367085e-02,  -1.26700244e-02,
          4.39036150e-03,  -1.01

In [170]:
ql.q.eval(feed_dict)

array([ 1746.92163086,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,  3156.87939453], dtype=float32)

In [174]:
ql.q_s1.eval(feed_dict)

array([ 10.50465488,  10.50465488,  10.50465488,  10.50465488,
        10.50465488,  10.50465488,  10.50465488,  10.50465488,
        10.50465488,  10.50465488], dtype=float32)

In [173]:
sess.run(ql.copy_op)

In [138]:
Select(ql.pred_vars[2], ql.action).eval(feed_dict)

array([ 2186.74389648,  6496.70410156,  2797.15771484,  4662.953125  ,
           0.        ,  2761.18579102,  3284.98608398,  6556.95996094,
        2748.39379883,     0.        ], dtype=float32)

In [148]:
tf.reduce_sum(ql.pred_vars[2] * tf.one_hot(ql.action, ql.num_actions), reduction_indices=1).eval(feed_dict)

array([ 2186.74389648,  6496.70410156,  2797.15771484,  4662.953125  ,
           0.        ,  2761.18579102,  3284.98608398,  6556.95996094,
        2748.39379883,     0.        ], dtype=float32)

In [93]:
tf.shape(ql.pred_vars[2]).eval(feed_dict)

array([10,  7], dtype=int32)

In [51]:
ql.delta.eval()

array([ 25.,  25.,  25.,  25.,  25.,  25.,  25.,  25.,  25.,  25.], dtype=float32)

In [40]:
tf.reduce_mean(ql.delta).eval({ql.state: ss, ql.action: aa, ql.reward: rr, ql.state1:ss1, ql.gamma: gg})

25.0

In [283]:
ql.q.eval({ql.state: ss, ql.action: aa})

array([    0.        ,     0.        ,  2561.84985352,  2696.66064453,
        3319.86474609,  2376.26171875,  2375.80859375,  2602.42016602,
        2548.83056641,     0.        ], dtype=float32)

In [284]:
ql.q_s1.eval({ql.state1: ss1})

array([ 9.87346268,  9.87346268,  9.87346268,  9.87346268,  9.87346268,
        9.87346268,  9.87346268,  9.87346268,  9.87346268,  9.87346268], dtype=float32)